# Overview

Data Source: https://www.kaggle.com/datasets/drgilermo/nba-players-stats

This dataset provides detailed game-by-game statistics and player salary information since 1996.

There are four distinct datasets:
1. boxscore: Contains player-specific game-by-game box scores.
2. games: Encompasses the game results for each match.
3. player_info: Includes demographic data for each player.
4. salaries: Presents player salaries, adjusted for inflation on an annual basis.


This notebook focuses on data exploration and preprocessing tasks before diving into any analysis. The primary objective is to leverage the provided data to predict player salaries.

The data is structured to showcase player performance per game, incorporating various metrics such as points per game, +/- per game, winning percentage, and more.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
box = pd.read_csv('/Users/rickykuo/rickykuo81704@gmail.com - Google Drive/My Drive/Jupyter/NBA Data/boxscore.csv')
games = pd.read_csv('/Users/rickykuo/rickykuo81704@gmail.com - Google Drive/My Drive/Jupyter/NBA Data/games.csv')
players = pd.read_csv('/Users/rickykuo/rickykuo81704@gmail.com - Google Drive/My Drive/Jupyter/NBA Data/player_info.csv')
salaries = pd.read_csv('/Users/rickykuo/rickykuo81704@gmail.com - Google Drive/My Drive/Jupyter/NBA Data/salaries.csv')

In [3]:
box.head()

,game_id,teamName,playerName,MP,FG,FGA,3P,3PA,FT,FTA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,isStarter
0,1,Sacramento Kings,Corliss Williamson,37:20,7,11,0,0,0,0,...,3,4,4,1,1,4,5,14,-2,1
1,1,Sacramento Kings,Mitch Richmond,32:00,6,12,1,4,1,1,...,5,5,3,1,2,3,1,14,-12,1
2,1,Sacramento Kings,Olden Polynice,31:34,0,4,0,0,1,4,...,5,7,3,1,0,4,3,1,-12,1
3,1,Sacramento Kings,Mahmoud Abdul-Rauf,29:27,7,13,1,2,2,2,...,2,2,5,1,1,2,2,17,-7,1
4,1,Sacramento Kings,Brian Grant,25:13,3,11,0,0,2,2,...,5,6,0,0,2,1,2,8,-7,1


In [4]:
games.head()

,seasonStartYear,awayTeam,pointsAway,homeTeam,pointsHome,attendance,notes,startET,datetime,isRegular,game_id
0,1996,Sacramento Kings,85,Houston Rockets,96,16285.0,NaN,NaN,1996-11-01,1,1
1,1996,Los Angeles Clippers,97,Golden State Warriors,85,15593.0,NaN,NaN,1996-11-01,1,2
2,1996,Portland Trail Blazers,114,Vancouver Grizzlies,85,19193.0,NaN,NaN,1996-11-01,1,3
3,1996,Seattle SuperSonics,91,Utah Jazz,99,19911.0,NaN,NaN,1996-11-01,1,4
4,1996,New York Knicks,107,Toronto Raptors,99,28457.0,NaN,NaN,1996-11-01,1,5


In [5]:
players.head()

,playerName,From,To,Pos,Ht,Wt,birthDate,Colleges
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State
2,Kareem Abdul-Jabbar*,1970,1989,C,7-2,225.0,"April 16, 1947",UCLA
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",LSU
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,"November 3, 1974","Michigan, San Jose State"


In [6]:
salaries.head()

,playerName,seasonStartYear,salary,inflationAdjSalary
0,Michael Jordan,1996,"$30,140,000","$52,258,566"
1,Horace Grant,1996,"$14,857,000","$25,759,971"
2,Reggie Miller,1996,"$11,250,000","$19,505,934"
3,Shaquille O'Neal,1996,"$10,714,000","$18,576,585"
4,Gary Payton,1996,"$10,212,000","$17,706,187"


In [7]:
# Get everything into one table
df = box.merge(games, left_on = 'game_id', right_on = 'game_id')
df = df.merge(salaries, on = ['playerName','seasonStartYear'])
df.columns

Index(['game_id', 'teamName', 'playerName', 'MP', 'FG', 'FGA', '3P', '3PA',
       'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', '+/-', 'isStarter', 'seasonStartYear', 'awayTeam', 'pointsAway',
       'homeTeam', 'pointsHome', 'attendance', 'notes', 'startET', 'datetime',
       'isRegular', 'salary', 'inflationAdjSalary'],
      dtype='object')

In [8]:
#See the % of NaN by field
df.isna().mean()

game_id               0.000000
teamName              0.000000
playerName            0.000000
MP                    0.000000
FG                    0.000000
FGA                   0.000000
3P                    0.000000
3PA                   0.000000
FT                    0.000000
FTA                   0.000000
ORB                   0.000000
DRB                   0.000000
TRB                   0.000000
AST                   0.000000
STL                   0.000000
BLK                   0.000000
TOV                   0.000000
PF                    0.000000
PTS                   0.000000
+/-                   0.000149
isStarter             0.000000
seasonStartYear       0.000000
awayTeam              0.000000
pointsAway            0.000000
homeTeam              0.000000
pointsHome            0.000000
attendance            0.005635
notes                 0.996447
startET               0.154107
datetime              0.000000
isRegular             0.000000
salary                0.000000
inflatio

In [9]:
# Quick glance at the data
df[~df['notes'].isna()]

,game_id,teamName,playerName,MP,FG,FGA,3P,3PA,FT,FTA,...,pointsAway,homeTeam,pointsHome,attendance,notes,startET,datetime,isRegular,salary,inflationAdjSalary
1630,211,Los Angeles Clippers,Loy Vaught,33:02,3,14,0,0,0,0,...,106,Los Angeles Clippers,95,11519.0,"at Anaheim, CA",NaN,1996-11-30,1,"$3,100,000","$5,374,968"
1638,322,Los Angeles Clippers,Loy Vaught,45:56,8,16,0,0,0,0,...,121,Los Angeles Clippers,122,12049.0,"at Anaheim, CA",NaN,1996-12-16,1,"$3,100,000","$5,374,968"
1653,577,Los Angeles Clippers,Loy Vaught,13:58,0,4,0,0,0,0,...,100,Los Angeles Clippers,102,16227.0,"at Anaheim, CA",NaN,1997-01-23,1,"$3,100,000","$5,374,968"
1658,666,Los Angeles Clippers,Loy Vaught,28:32,10,11,0,0,2,2,...,86,Los Angeles Clippers,108,18462.0,"at Anaheim, CA",NaN,1997-02-04,1,"$3,100,000","$5,374,968"
1671,857,Los Angeles Clippers,Loy Vaught,48:00,13,20,0,0,2,2,...,94,Los Angeles Clippers,93,14916.0,"at Anaheim, CA",NaN,1997-03-06,1,"$3,100,000","$5,374,968"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667926,30167,Memphis Grizzlies,John Konchar,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,...,122,Portland Trail Blazers,126,NaN,Play-In Game,2:30p,2020-08-15,0,"$79,568","$84,399"
668226,29476,Detroit Pistons,Blake Griffin,29:39,3,16,1,10,3,4,...,122,Detroit Pistons,111,20064.0,"at Mexico City, Mexico",9:00p,2019-12-12,1,"$34,449,964","$36,541,765"
668747,29776,Milwaukee Bucks,Dragan Bender,5:39,0,1,0,0,0,0,...,116,Charlotte Hornets,103,15758.0,"at Paris, France",3:00p,2020-01-24,1,"$189,700","$201,218"
668801,29493,Phoenix Suns,Ty Jerome,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,...,121,Phoenix Suns,119,20013.0,"at Mexico City, Mexico",5:00p,2019-12-14,1,"$2,193,480","$2,326,668"


In [10]:
# Quick glance at the data
df[~df['startET'].isna()]

,game_id,teamName,playerName,MP,FG,FGA,3P,3PA,FT,FTA,...,pointsAway,homeTeam,pointsHome,attendance,notes,startET,datetime,isRegular,salary,inflationAdjSalary
103188,4577,Seattle SuperSonics,Gary Payton,48:00,10,26,2,9,5,7,...,88,Vancouver Grizzlies,94,15779.0,NaN,10:30p,2000-10-31,1,"$12,200,000","$19,226,747"
103189,4595,Seattle SuperSonics,Gary Payton,34:45,13,16,1,1,8,10,...,99,Seattle SuperSonics,112,17072.0,NaN,10:00p,2000-11-01,1,"$12,200,000","$19,226,747"
103190,4615,Seattle SuperSonics,Gary Payton,48:00,9,20,1,4,6,8,...,97,Seattle SuperSonics,90,17072.0,NaN,10:00p,2000-11-04,1,"$12,200,000","$19,226,747"
103191,4629,Seattle SuperSonics,Gary Payton,41:45,8,15,2,6,5,6,...,99,Orlando Magic,110,12460.0,NaN,7:30p,2000-11-06,1,"$12,200,000","$19,226,747"
103192,4643,Seattle SuperSonics,Gary Payton,43:21,9,20,3,5,1,2,...,81,Miami Heat,87,15083.0,NaN,7:30p,2000-11-08,1,"$12,200,000","$19,226,747"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669582,30198,Portland Trail Blazers,Jaylen Adams,8:55,0,3,0,2,0,0,...,135,Portland Trail Blazers,115,NaN,NaN,9:00p,2020-08-24,0,"$163,356","$173,274"
669583,30204,Portland Trail Blazers,Jaylen Adams,1:56,1,2,0,0,0,0,...,122,Los Angeles Lakers,131,NaN,NaN,9:00p,2020-08-29,0,"$163,356","$173,274"
669584,30108,Brooklyn Nets,Jamal Crawford,5:58,2,4,1,2,0,0,...,119,Milwaukee Bucks,116,NaN,NaN,1:30p,2020-08-04,1,"$289,803","$307,399"
669585,30145,New Orleans Pelicans,Sindarius Thornwell,16:05,3,5,1,1,0,0,...,106,Sacramento Kings,112,NaN,NaN,9:00p,2020-08-11,1,"$183,115","$194,233"


In [11]:
#Remove unwanted fields
columns = ['attendance','notes','startET','datetime','isRegular','salary']
df = df.drop(columns,axis = 1)
df

,game_id,teamName,playerName,MP,FG,FGA,3P,3PA,FT,FTA,...,PF,PTS,+/-,isStarter,seasonStartYear,awayTeam,pointsAway,homeTeam,pointsHome,inflationAdjSalary
0,1,Sacramento Kings,Corliss Williamson,37:20,7,11,0,0,0,0,...,5,14,-2,1,1996,Sacramento Kings,85,Houston Rockets,96,"$1,822,287"
1,18,Sacramento Kings,Corliss Williamson,22:41,3,7,0,0,1,3,...,4,7,+9,1,1996,Sacramento Kings,107,Dallas Mavericks,94,"$1,822,287"
2,38,Sacramento Kings,Corliss Williamson,26:12,6,12,0,0,2,3,...,4,14,+1,1,1996,Houston Rockets,102,Sacramento Kings,80,"$1,822,287"
3,47,Sacramento Kings,Corliss Williamson,22:09,1,4,0,0,1,2,...,4,3,-3,1,1996,Atlanta Hawks,91,Sacramento Kings,87,"$1,822,287"
4,64,Sacramento Kings,Corliss Williamson,20:52,4,8,0,0,6,6,...,0,14,-5,0,1996,Portland Trail Blazers,102,Sacramento Kings,103,"$1,822,287"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669582,30198,Portland Trail Blazers,Jaylen Adams,8:55,0,3,0,2,0,0,...,0,0,+9,0,2019,Los Angeles Lakers,135,Portland Trail Blazers,115,"$173,274"
669583,30204,Portland Trail Blazers,Jaylen Adams,1:56,1,2,0,0,0,0,...,1,2,+6,0,2019,Portland Trail Blazers,122,Los Angeles Lakers,131,"$173,274"
669584,30108,Brooklyn Nets,Jamal Crawford,5:58,2,4,1,2,0,0,...,1,5,+4,0,2019,Brooklyn Nets,119,Milwaukee Bucks,116,"$307,399"
669585,30145,New Orleans Pelicans,Sindarius Thornwell,16:05,3,5,1,1,0,0,...,1,7,-3,0,2019,New Orleans Pelicans,106,Sacramento Kings,112,"$194,233"


In [12]:
#Add a binary field named homeGame to show whether the game is played at home
df['homeGame'] = df['teamName'] == df['homeTeam']
df

,game_id,teamName,playerName,MP,FG,FGA,3P,3PA,FT,FTA,...,PTS,+/-,isStarter,seasonStartYear,awayTeam,pointsAway,homeTeam,pointsHome,inflationAdjSalary,homeGame
0,1,Sacramento Kings,Corliss Williamson,37:20,7,11,0,0,0,0,...,14,-2,1,1996,Sacramento Kings,85,Houston Rockets,96,"$1,822,287",False
1,18,Sacramento Kings,Corliss Williamson,22:41,3,7,0,0,1,3,...,7,+9,1,1996,Sacramento Kings,107,Dallas Mavericks,94,"$1,822,287",False
2,38,Sacramento Kings,Corliss Williamson,26:12,6,12,0,0,2,3,...,14,+1,1,1996,Houston Rockets,102,Sacramento Kings,80,"$1,822,287",True
3,47,Sacramento Kings,Corliss Williamson,22:09,1,4,0,0,1,2,...,3,-3,1,1996,Atlanta Hawks,91,Sacramento Kings,87,"$1,822,287",True
4,64,Sacramento Kings,Corliss Williamson,20:52,4,8,0,0,6,6,...,14,-5,0,1996,Portland Trail Blazers,102,Sacramento Kings,103,"$1,822,287",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669582,30198,Portland Trail Blazers,Jaylen Adams,8:55,0,3,0,2,0,0,...,0,+9,0,2019,Los Angeles Lakers,135,Portland Trail Blazers,115,"$173,274",True
669583,30204,Portland Trail Blazers,Jaylen Adams,1:56,1,2,0,0,0,0,...,2,+6,0,2019,Portland Trail Blazers,122,Los Angeles Lakers,131,"$173,274",False
669584,30108,Brooklyn Nets,Jamal Crawford,5:58,2,4,1,2,0,0,...,5,+4,0,2019,Brooklyn Nets,119,Milwaukee Bucks,116,"$307,399",False
669585,30145,New Orleans Pelicans,Sindarius Thornwell,16:05,3,5,1,1,0,0,...,7,-3,0,2019,New Orleans Pelicans,106,Sacramento Kings,112,"$194,233",False


In [13]:
#Create a method to check the result of each game by each player
def win(homeGame,pointsHome,pointsAway):
    if (homeGame == True) & (pointsHome > pointsAway):
        return "W"
    elif (homeGame == True) & (pointsHome < pointsAway):
        return "L"        
    elif (homeGame == False) & (pointsHome < pointsAway):
        return "W"
    else:
        return "L"
    

In [15]:
#Test the method
win(False, 101,100)

'L'

In [16]:
#Deploy the method
df['result'] = df.apply(lambda row: win(row['homeGame'], row['pointsHome'], row['pointsAway']), axis=1)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 669587 entries, 0 to 669586
Data columns (total 29 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   game_id             669587 non-null  int64 
 1   teamName            669587 non-null  object
 2   playerName          669587 non-null  object
 3   MP                  669587 non-null  object
 4   FG                  669587 non-null  object
 5   FGA                 669587 non-null  object
 6   3P                  669587 non-null  object
 7   3PA                 669587 non-null  object
 8   FT                  669587 non-null  object
 9   FTA                 669587 non-null  object
 10  ORB                 669587 non-null  object
 11  DRB                 669587 non-null  object
 12  TRB                 669587 non-null  object
 13  AST                 669587 non-null  object
 14  STL                 669587 non-null  object
 15  BLK                 669587 non-null  object
 16  TO

In [18]:
columns = ['FG', 'FGA', '3P', '3PA','FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF','PTS', '+/-','inflationAdjSalary']

In [19]:
df[columns] 

,FG,FGA,3P,3PA,FT,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,inflationAdjSalary
0,7,11,0,0,0,0,1,3,4,4,1,1,4,5,14,-2,"$1,822,287"
1,3,7,0,0,1,3,2,1,3,1,1,0,3,4,7,+9,"$1,822,287"
2,6,12,0,0,2,3,2,4,6,1,1,0,6,4,14,+1,"$1,822,287"
3,1,4,0,0,1,2,2,4,6,2,0,0,1,4,3,-3,"$1,822,287"
4,4,8,0,0,6,6,1,0,1,1,1,0,0,0,14,-5,"$1,822,287"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669582,0,3,0,2,0,0,1,0,1,0,1,0,0,0,0,+9,"$173,274"
669583,1,2,0,0,0,0,0,0,0,0,0,0,0,1,2,+6,"$173,274"
669584,2,4,1,2,0,0,0,0,0,3,0,0,0,1,5,+4,"$307,399"
669585,3,5,1,1,0,0,0,2,2,3,1,0,0,1,7,-3,"$194,233"


In [20]:
#Check distinct inputs of each field
for col in df[columns]:
    print(col)
    print(df[col].unique())

FG
['7' '3' '6' '1' '4' '0' '2' '9' '5' '8' 'Not With Team' 'Did Not Dress'
 '11' '10' '12' '13' '15' '14' 'Did Not Play' 'Player Suspended' '18' '24'
 '16' '17' '19' '22' '21' '20' '23' '28']
FGA
['11' '7' '12' '4' '8' '2' '6' '16' '3' '9' '13' '10' '15' 'Not With Team'
 'Did Not Dress' '14' '5' '21' '18' '23' '20' '24' '27' '17' '26' '28'
 '30' '25' '19' '22' 'Did Not Play' '1' '0' 'Player Suspended' '35' '29'
 '40' '31' '37' '32' '33' '34' '36' '39' '38' '41' '47' '42' '46' '45'
 '44' '43' '50']
3P
['0' 'Not With Team' 'Did Not Dress' '1' '2' '3' '4' '5' '7' '6'
 'Did Not Play' 'Player Suspended' '8' '9' '10' '12' '11' '13' '14']
3PA
['0' '1' 'Not With Team' 'Did Not Dress' '4' '2' '3' '6' '5' '7' '8' '9'
 '10' '12' '11' 'Did Not Play' 'Player Suspended' '13' '14' '15' '17' '16'
 '18' '19' '21' '20' '24' '23' '22']
FT
['0' '1' '2' '6' '3' '4' '5' 'Not With Team' 'Did Not Dress' '7' '11' '9'
 '14' '8' '12' '13' '15' '10' 'Did Not Play' 'Player Suspended' '16' '17'
 '22' '18' '19' '23

In [21]:
#remove unwanted strings in the fields
words = ['Not With Team','Did Not Dress','Did Not Play','Player Suspended']
df = df.replace(words,0)

In [22]:
df['inflationAdjSalary'] = df['inflationAdjSalary'].str.replace('$','')
df['inflationAdjSalary'] = df['inflationAdjSalary'].str.replace(',','')

/var/folders/4n/n71rnzzj6w5fbx03g8r_hj9r0000gp/T/ipykernel_67598/1983831617.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['inflationAdjSalary'] = df['inflationAdjSalary'].str.replace('$','')


In [25]:
#Fill NaN with 0
df = df.fillna(0)
df

,game_id,teamName,playerName,MP,FG,FGA,3P,3PA,FT,FTA,...,+/-,isStarter,seasonStartYear,awayTeam,pointsAway,homeTeam,pointsHome,inflationAdjSalary,homeGame,result
0,1,Sacramento Kings,Corliss Williamson,37:20,7,11,0,0,0,0,...,-2,1,1996,Sacramento Kings,85,Houston Rockets,96,1822287,False,L
1,18,Sacramento Kings,Corliss Williamson,22:41,3,7,0,0,1,3,...,+9,1,1996,Sacramento Kings,107,Dallas Mavericks,94,1822287,False,W
2,38,Sacramento Kings,Corliss Williamson,26:12,6,12,0,0,2,3,...,+1,1,1996,Houston Rockets,102,Sacramento Kings,80,1822287,True,L
3,47,Sacramento Kings,Corliss Williamson,22:09,1,4,0,0,1,2,...,-3,1,1996,Atlanta Hawks,91,Sacramento Kings,87,1822287,True,L
4,64,Sacramento Kings,Corliss Williamson,20:52,4,8,0,0,6,6,...,-5,0,1996,Portland Trail Blazers,102,Sacramento Kings,103,1822287,True,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669582,30198,Portland Trail Blazers,Jaylen Adams,8:55,0,3,0,2,0,0,...,+9,0,2019,Los Angeles Lakers,135,Portland Trail Blazers,115,173274,True,L
669583,30204,Portland Trail Blazers,Jaylen Adams,1:56,1,2,0,0,0,0,...,+6,0,2019,Portland Trail Blazers,122,Los Angeles Lakers,131,173274,False,L
669584,30108,Brooklyn Nets,Jamal Crawford,5:58,2,4,1,2,0,0,...,+4,0,2019,Brooklyn Nets,119,Milwaukee Bucks,116,307399,False,W
669585,30145,New Orleans Pelicans,Sindarius Thornwell,16:05,3,5,1,1,0,0,...,-3,0,2019,New Orleans Pelicans,106,Sacramento Kings,112,194233,False,L


In [26]:
#convert all columns to type int
df[columns] = df[columns].astype(int)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 669587 entries, 0 to 669586
Data columns (total 29 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   game_id             669587 non-null  int64 
 1   teamName            669587 non-null  object
 2   playerName          669587 non-null  object
 3   MP                  669587 non-null  object
 4   FG                  669587 non-null  int64 
 5   FGA                 669587 non-null  int64 
 6   3P                  669587 non-null  int64 
 7   3PA                 669587 non-null  int64 
 8   FT                  669587 non-null  int64 
 9   FTA                 669587 non-null  int64 
 10  ORB                 669587 non-null  int64 
 11  DRB                 669587 non-null  int64 
 12  TRB                 669587 non-null  int64 
 13  AST                 669587 non-null  int64 
 14  STL                 669587 non-null  int64 
 15  BLK                 669587 non-null  int64 
 16  TO

In [28]:
# Transform playing time to minutes
def MP(MP):
    if MP == 0:
        return 0
    else:
        M = MP.split(":")[0]
        S = MP.split(":")[-1]
    return int(M)*60 + int(S)

In [29]:
#Test the method
MP('22:41')

1361

In [35]:

df['seconds'] = df['MP'].apply(MP)

In [36]:
df['MP'] = df['seconds']/60

In [37]:
df.columns

Index(['game_id', 'teamName', 'playerName', 'MP', 'FG', 'FGA', '3P', '3PA',
       'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', '+/-', 'isStarter', 'seasonStartYear', 'awayTeam', 'pointsAway',
       'homeTeam', 'pointsHome', 'inflationAdjSalary', 'homeGame', 'result',
       'seconds'],
      dtype='object')

In [38]:
df = df.drop(['seconds','Minutes'],axis = 1)

KeyError: "['Minutes'] not found in axis"

In [39]:
df.columns

Index(['game_id', 'teamName', 'playerName', 'MP', 'FG', 'FGA', '3P', '3PA',
       'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', '+/-', 'isStarter', 'seasonStartYear', 'awayTeam', 'pointsAway',
       'homeTeam', 'pointsHome', 'inflationAdjSalary', 'homeGame', 'result',
       'seconds'],
      dtype='object')

In [41]:
df['game_id'] = df['game_id'].astype(str)

In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 669587 entries, 0 to 669586
Data columns (total 30 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   game_id             669587 non-null  int64  
 1   teamName            669587 non-null  object 
 2   playerName          669587 non-null  object 
 3   MP                  669587 non-null  float64
 4   FG                  669587 non-null  int64  
 5   FGA                 669587 non-null  int64  
 6   3P                  669587 non-null  int64  
 7   3PA                 669587 non-null  int64  
 8   FT                  669587 non-null  int64  
 9   FTA                 669587 non-null  int64  
 10  ORB                 669587 non-null  int64  
 11  DRB                 669587 non-null  int64  
 12  TRB                 669587 non-null  int64  
 13  AST                 669587 non-null  int64  
 14  STL                 669587 non-null  int64  
 15  BLK                 669587 non-nul

In [150]:
per_player_stats = df.groupby('playerName').mean()

In [151]:
per_player_stats

,game_id,MP,FG,FGA,3P,3PA,FT,FTA,ORB,DRB,...,PF,PTS,+/-,isStarter,seasonStartYear,pointsAway,pointsHome,inflationAdjSalary,homeGame,seconds
playerName,,,,,,,,,,,,,,,,,,,,,
A.C. Green,2988.748148,24.350823,2.318519,5.130864,0.004938,0.103704,1.076543,1.582716,2.059259,3.992593,...,1.641975,5.718519,-1.037037,0.696296,1998.066667,91.940741,95.111111,5.991571e+06,0.511111,1461.049383
A.J. Bramlett,3637.526316,3.201754,0.210526,1.105263,0.000000,0.000000,0.000000,0.000000,0.631579,0.526316,...,0.684211,0.421053,-2.210526,0.000000,1999.000000,98.105263,104.052632,1.944930e+05,0.578947,192.105263
Aaron Brooks,21002.756127,17.330739,2.921569,7.075980,1.050245,2.855392,1.188725,1.428922,0.323529,1.082108,...,1.563725,8.082108,-0.375000,0.240196,2012.250000,100.359069,103.563725,2.350954e+06,0.498775,1039.844363
Aaron Gordon,26530.383372,26.975520,4.547344,10.143187,1.036952,3.240185,1.921478,2.752887,1.501155,4.556582,...,1.921478,12.053118,-1.221709,0.736721,2016.565820,103.441109,105.639723,1.091840e+07,0.510393,1618.531178
Aaron Gray,17493.515152,8.002370,0.934732,1.839161,0.000000,0.006993,0.386946,0.687646,0.899767,1.615385,...,1.412587,2.256410,-0.622378,0.165501,2009.494172,97.934732,100.808858,1.577036e+06,0.494172,480.142191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zhaire Smith,29309.727273,6.746212,0.772727,2.045455,0.272727,0.863636,0.363636,0.545455,0.136364,0.545455,...,0.545455,2.181818,0.772727,0.090909,2018.454545,106.954545,115.909091,3.012794e+06,0.409091,404.772727
Zhou Qi,27041.590909,2.970076,0.181818,0.772727,0.045455,0.431818,0.181818,0.272727,0.136364,0.386364,...,0.318182,0.590909,-1.568182,0.000000,2017.045455,107.477273,109.727273,9.310791e+05,0.613636,178.204545
Zion Williamson,29952.080000,26.725333,8.400000,14.400000,0.240000,0.560000,4.560000,7.120000,2.560000,3.440000,...,1.680000,21.600000,2.480000,0.960000,2019.000000,116.600000,117.880000,1.034991e+07,0.560000,1603.520000


In [152]:
per_player_stats.columns

Index(['game_id', 'MP', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'DRB',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-', 'isStarter',
       'seasonStartYear', 'pointsAway', 'pointsHome', 'inflationAdjSalary',
       'homeGame', 'seconds'],
      dtype='object')

In [153]:
per_player_stats = per_player_stats.drop(['game_id','FG', 'FGA', '3P', '3PA', 'FT', 'FTA','seasonStartYear', 'pointsAway', 'pointsHome','homeGame', 'seconds'],axis = 1)

In [154]:
per_player_stats

,MP,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,isStarter,inflationAdjSalary
playerName,,,,,,,,,,,,,
A.C. Green,24.350823,2.059259,3.992593,6.051852,0.866667,0.676543,0.197531,0.664198,1.641975,5.718519,-1.037037,0.696296,5.991571e+06
A.J. Bramlett,3.201754,0.631579,0.526316,1.157895,0.000000,0.052632,0.000000,0.157895,0.684211,0.421053,-2.210526,0.000000,1.944930e+05
Aaron Brooks,17.330739,0.323529,1.082108,1.405637,2.444853,0.450980,0.112745,1.284314,1.563725,8.082108,-0.375000,0.240196,2.350954e+06
Aaron Gordon,26.975520,1.501155,4.556582,6.057737,2.267898,0.734411,0.595843,1.327945,1.921478,12.053118,-1.221709,0.736721,1.091840e+07
Aaron Gray,8.002370,0.899767,1.615385,2.515152,0.410256,0.207459,0.195804,0.547786,1.412587,2.256410,-0.622378,0.165501,1.577036e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zhaire Smith,6.746212,0.136364,0.545455,0.681818,0.545455,0.272727,0.090909,0.363636,0.545455,2.181818,0.772727,0.090909,3.012794e+06
Zhou Qi,2.970076,0.136364,0.386364,0.522727,0.045455,0.045455,0.318182,0.227273,0.318182,0.590909,-1.568182,0.000000,9.310791e+05
Zion Williamson,26.725333,2.560000,3.440000,6.000000,2.000000,0.640000,0.360000,2.360000,1.680000,21.600000,2.480000,0.960000,1.034991e+07


In [155]:
wins = df.groupby(['playerName','result']).count()['game_id']

In [156]:
wins.head()

playerName     result
A.C. Green     L         211
               W         194
A.J. Bramlett  L          13
               W           6
Aaron Brooks   L         372
Name: game_id, dtype: int64

In [157]:
wins.index

MultiIndex([(        'A.C. Green', 'L'),
            (        'A.C. Green', 'W'),
            (     'A.J. Bramlett', 'L'),
            (     'A.J. Bramlett', 'W'),
            (      'Aaron Brooks', 'L'),
            (      'Aaron Brooks', 'W'),
            (      'Aaron Gordon', 'L'),
            (      'Aaron Gordon', 'W'),
            (        'Aaron Gray', 'L'),
            (        'Aaron Gray', 'W'),
            ...
            (      'Zhaire Smith', 'L'),
            (      'Zhaire Smith', 'W'),
            (           'Zhou Qi', 'L'),
            (           'Zhou Qi', 'W'),
            (   'Zion Williamson', 'L'),
            (   'Zion Williamson', 'W'),
            ('Zydrunas Ilgauskas', 'L'),
            ('Zydrunas Ilgauskas', 'W'),
            (    'Zylan Cheatham', 'L'),
            (    'Zylan Cheatham', 'W')],
           names=['playerName', 'result'], length=3898)

In [158]:
wins = wins.xs(key='W',axis=0,level='result')

In [159]:
wins = pd.DataFrame(wins)

In [160]:
wins

,game_id
playerName,
A.C. Green,194
A.J. Bramlett,6
Aaron Brooks,444
Aaron Gordon,173
Aaron Gray,180
...,...
Zhaire Smith,10
Zhou Qi,30
Zion Williamson,12


In [161]:
wins['total_wins'] = wins['game_id']
wins = wins.drop('game_id',axis = 1)

In [162]:
wins

,total_wins
playerName,
A.C. Green,194
A.J. Bramlett,6
Aaron Brooks,444
Aaron Gordon,173
Aaron Gray,180
...,...
Zhaire Smith,10
Zhou Qi,30
Zion Williamson,12


In [163]:
per_player_stats = per_player_stats.merge(wins, on='playerName')

In [164]:
per_player_stats.columns

Index(['MP', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       '+/-', 'isStarter', 'inflationAdjSalary', 'total_wins'],
      dtype='object')

In [165]:
per_player_stats

,MP,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,isStarter,inflationAdjSalary,total_wins
playerName,,,,,,,,,,,,,,
A.C. Green,24.350823,2.059259,3.992593,6.051852,0.866667,0.676543,0.197531,0.664198,1.641975,5.718519,-1.037037,0.696296,5.991571e+06,194
A.J. Bramlett,3.201754,0.631579,0.526316,1.157895,0.000000,0.052632,0.000000,0.157895,0.684211,0.421053,-2.210526,0.000000,1.944930e+05,6
Aaron Brooks,17.330739,0.323529,1.082108,1.405637,2.444853,0.450980,0.112745,1.284314,1.563725,8.082108,-0.375000,0.240196,2.350954e+06,444
Aaron Gordon,26.975520,1.501155,4.556582,6.057737,2.267898,0.734411,0.595843,1.327945,1.921478,12.053118,-1.221709,0.736721,1.091840e+07,173
Aaron Gray,8.002370,0.899767,1.615385,2.515152,0.410256,0.207459,0.195804,0.547786,1.412587,2.256410,-0.622378,0.165501,1.577036e+06,180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zhaire Smith,6.746212,0.136364,0.545455,0.681818,0.545455,0.272727,0.090909,0.363636,0.545455,2.181818,0.772727,0.090909,3.012794e+06,10
Zhou Qi,2.970076,0.136364,0.386364,0.522727,0.045455,0.045455,0.318182,0.227273,0.318182,0.590909,-1.568182,0.000000,9.310791e+05,30
Zion Williamson,26.725333,2.560000,3.440000,6.000000,2.000000,0.640000,0.360000,2.360000,1.680000,21.600000,2.480000,0.960000,1.034991e+07,12


In [166]:
shooting = df.groupby('playerName').sum()

In [167]:
shooting

,game_id,MP,FG,FGA,3P,3PA,FT,FTA,ORB,DRB,...,PF,PTS,+/-,isStarter,seasonStartYear,pointsAway,pointsHome,inflationAdjSalary,homeGame,seconds
playerName,,,,,,,,,,,,,,,,,,,,,
A.C. Green,1210443,9862.083333,939,2078,2,42,436,641,834,1617,...,665,2316,-420,282,809217,37236,38520,2426586239,207,591725
A.J. Bramlett,69113,60.833333,4,21,0,0,0,0,12,10,...,13,8,-42,0,37981,1864,1977,3695367,11,3650
Aaron Brooks,17138249,14141.883333,2384,5774,857,2330,970,1166,264,883,...,1276,6595,-306,196,1641996,81893,84508,1918378541,407,848513
Aaron Gordon,11487656,11680.400000,1969,4392,449,1403,832,1192,650,1973,...,832,5219,-529,319,873173,44790,45742,4727667539,221,700824
Aaron Gray,7504718,3433.016667,401,789,0,3,166,295,386,693,...,606,968,-267,71,862073,42014,43247,676548410,212,205981
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zhaire Smith,644814,148.416667,17,45,6,19,8,12,3,12,...,12,48,17,2,44406,2353,2550,66281460,9,8905
Zhou Qi,1189830,130.683333,8,34,2,19,8,12,6,17,...,14,26,-69,0,88750,4729,4828,40967482,27,7841
Zion Williamson,748802,668.133333,210,360,6,14,114,178,64,86,...,42,540,62,24,50475,2915,2947,258747775,14,40088


In [168]:
shooting['FGP'] = shooting['FG'] / shooting['FGA']
shooting['3PP'] = shooting['3P'] / shooting['3PA']
shooting['FTP'] = shooting['FT'] / shooting['FTA']

In [169]:
col = ['MP','FGP','3PP','FTP']
shooting = shooting[col]

In [170]:
per_player_stats

,MP,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,isStarter,inflationAdjSalary,total_wins
playerName,,,,,,,,,,,,,,
A.C. Green,24.350823,2.059259,3.992593,6.051852,0.866667,0.676543,0.197531,0.664198,1.641975,5.718519,-1.037037,0.696296,5.991571e+06,194
A.J. Bramlett,3.201754,0.631579,0.526316,1.157895,0.000000,0.052632,0.000000,0.157895,0.684211,0.421053,-2.210526,0.000000,1.944930e+05,6
Aaron Brooks,17.330739,0.323529,1.082108,1.405637,2.444853,0.450980,0.112745,1.284314,1.563725,8.082108,-0.375000,0.240196,2.350954e+06,444
Aaron Gordon,26.975520,1.501155,4.556582,6.057737,2.267898,0.734411,0.595843,1.327945,1.921478,12.053118,-1.221709,0.736721,1.091840e+07,173
Aaron Gray,8.002370,0.899767,1.615385,2.515152,0.410256,0.207459,0.195804,0.547786,1.412587,2.256410,-0.622378,0.165501,1.577036e+06,180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zhaire Smith,6.746212,0.136364,0.545455,0.681818,0.545455,0.272727,0.090909,0.363636,0.545455,2.181818,0.772727,0.090909,3.012794e+06,10
Zhou Qi,2.970076,0.136364,0.386364,0.522727,0.045455,0.045455,0.318182,0.227273,0.318182,0.590909,-1.568182,0.000000,9.310791e+05,30
Zion Williamson,26.725333,2.560000,3.440000,6.000000,2.000000,0.640000,0.360000,2.360000,1.680000,21.600000,2.480000,0.960000,1.034991e+07,12


In [171]:
per_player_stats = per_player_stats.merge(shooting, on='playerName')

In [176]:
# per_player_stats = per_player_stats.drop('MP_x',axis = 1)
per_player_stats['MP'] = per_player_stats['MP_y']
per_player_stats = per_player_stats.drop('MP_y',axis = 1)

In [177]:
per_player_stats.columns

Index(['ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-',
       'isStarter', 'inflationAdjSalary', 'total_wins', 'FGP', '3PP', 'FTP',
       'MP'],
      dtype='object')

In [180]:
games = df.groupby('playerName').count()['game_id']

In [181]:
games = pd.DataFrame(games)

In [184]:
games['games_played'] = games['game_id']
games

,game_id,games_played
playerName,,
A.C. Green,405,405
A.J. Bramlett,19,19
Aaron Brooks,816,816
Aaron Gordon,433,433
Aaron Gray,429,429
...,...,...
Zhaire Smith,22,22
Zhou Qi,44,44
Zion Williamson,25,25


In [185]:
per_player_stats = per_player_stats.merge(games, on='playerName')

In [186]:
per_player_stats.columns

Index(['ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-',
       'isStarter', 'inflationAdjSalary', 'total_wins', 'FGP', '3PP', 'FTP',
       'MP', 'game_id', 'games_played'],
      dtype='object')

In [188]:
per_player_stats = per_player_stats.drop('game_id', axis = 1)

In [190]:
per_player_stats.columns

Index(['ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-',
       'isStarter', 'inflationAdjSalary', 'total_wins', 'FGP', '3PP', 'FTP',
       'MP', 'games_played'],
      dtype='object')

In [192]:
per_player_stats

,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,isStarter,inflationAdjSalary,total_wins,FGP,3PP,FTP,MP,games_played
playerName,,,,,,,,,,,,,,,,,,
A.C. Green,2.059259,3.992593,6.051852,0.866667,0.676543,0.197531,0.664198,1.641975,5.718519,-1.037037,0.696296,5.991571e+06,194,0.451877,0.047619,0.680187,9862.083333,405
A.J. Bramlett,0.631579,0.526316,1.157895,0.000000,0.052632,0.000000,0.157895,0.684211,0.421053,-2.210526,0.000000,1.944930e+05,6,0.190476,NaN,NaN,60.833333,19
Aaron Brooks,0.323529,1.082108,1.405637,2.444853,0.450980,0.112745,1.284314,1.563725,8.082108,-0.375000,0.240196,2.350954e+06,444,0.412885,0.367811,0.831904,14141.883333,816
Aaron Gordon,1.501155,4.556582,6.057737,2.267898,0.734411,0.595843,1.327945,1.921478,12.053118,-1.221709,0.736721,1.091840e+07,173,0.448315,0.320029,0.697987,11680.400000,433
Aaron Gray,0.899767,1.615385,2.515152,0.410256,0.207459,0.195804,0.547786,1.412587,2.256410,-0.622378,0.165501,1.577036e+06,180,0.508238,0.000000,0.562712,3433.016667,429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zhaire Smith,0.136364,0.545455,0.681818,0.545455,0.272727,0.090909,0.363636,0.545455,2.181818,0.772727,0.090909,3.012794e+06,10,0.377778,0.315789,0.666667,148.416667,22
Zhou Qi,0.136364,0.386364,0.522727,0.045455,0.045455,0.318182,0.227273,0.318182,0.590909,-1.568182,0.000000,9.310791e+05,30,0.235294,0.105263,0.666667,130.683333,44
Zion Williamson,2.560000,3.440000,6.000000,2.000000,0.640000,0.360000,2.360000,1.680000,21.600000,2.480000,0.960000,1.034991e+07,12,0.583333,0.428571,0.640449,668.133333,25


In [193]:
per_player_stats['win_percentage'] = per_player_stats['total_wins']/per_player_stats['games_played']

In [196]:
per_player_stats['MPG'] = per_player_stats['MP']/per_player_stats['games_played']

In [197]:
per_player_stats

,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,isStarter,inflationAdjSalary,total_wins,FGP,3PP,FTP,MP,games_played,win_percentage,MPG
playerName,,,,,,,,,,,,,,,,,,,,
A.C. Green,2.059259,3.992593,6.051852,0.866667,0.676543,0.197531,0.664198,1.641975,5.718519,-1.037037,0.696296,5.991571e+06,194,0.451877,0.047619,0.680187,9862.083333,405,0.479012,24.350823
A.J. Bramlett,0.631579,0.526316,1.157895,0.000000,0.052632,0.000000,0.157895,0.684211,0.421053,-2.210526,0.000000,1.944930e+05,6,0.190476,NaN,NaN,60.833333,19,0.315789,3.201754
Aaron Brooks,0.323529,1.082108,1.405637,2.444853,0.450980,0.112745,1.284314,1.563725,8.082108,-0.375000,0.240196,2.350954e+06,444,0.412885,0.367811,0.831904,14141.883333,816,0.544118,17.330739
Aaron Gordon,1.501155,4.556582,6.057737,2.267898,0.734411,0.595843,1.327945,1.921478,12.053118,-1.221709,0.736721,1.091840e+07,173,0.448315,0.320029,0.697987,11680.400000,433,0.399538,26.975520
Aaron Gray,0.899767,1.615385,2.515152,0.410256,0.207459,0.195804,0.547786,1.412587,2.256410,-0.622378,0.165501,1.577036e+06,180,0.508238,0.000000,0.562712,3433.016667,429,0.419580,8.002370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zhaire Smith,0.136364,0.545455,0.681818,0.545455,0.272727,0.090909,0.363636,0.545455,2.181818,0.772727,0.090909,3.012794e+06,10,0.377778,0.315789,0.666667,148.416667,22,0.454545,6.746212
Zhou Qi,0.136364,0.386364,0.522727,0.045455,0.045455,0.318182,0.227273,0.318182,0.590909,-1.568182,0.000000,9.310791e+05,30,0.235294,0.105263,0.666667,130.683333,44,0.681818,2.970076
Zion Williamson,2.560000,3.440000,6.000000,2.000000,0.640000,0.360000,2.360000,1.680000,21.600000,2.480000,0.960000,1.034991e+07,12,0.583333,0.428571,0.640449,668.133333,25,0.480000,26.725333


In [210]:
per_player_stats = per_player_stats.fillna(0)

In [211]:
per_player_stats

,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,isStarter,inflationAdjSalary,total_wins,FGP,3PP,FTP,MP,games_played,win_percentage,MPG
playerName,,,,,,,,,,,,,,,,,,,,
A.C. Green,2.059259,3.992593,6.051852,0.866667,0.676543,0.197531,0.664198,1.641975,5.718519,-1.037037,0.696296,5.991571e+06,194,0.451877,0.047619,0.680187,9862.083333,405,0.479012,24.350823
A.J. Bramlett,0.631579,0.526316,1.157895,0.000000,0.052632,0.000000,0.157895,0.684211,0.421053,-2.210526,0.000000,1.944930e+05,6,0.190476,0.000000,0.000000,60.833333,19,0.315789,3.201754
Aaron Brooks,0.323529,1.082108,1.405637,2.444853,0.450980,0.112745,1.284314,1.563725,8.082108,-0.375000,0.240196,2.350954e+06,444,0.412885,0.367811,0.831904,14141.883333,816,0.544118,17.330739
Aaron Gordon,1.501155,4.556582,6.057737,2.267898,0.734411,0.595843,1.327945,1.921478,12.053118,-1.221709,0.736721,1.091840e+07,173,0.448315,0.320029,0.697987,11680.400000,433,0.399538,26.975520
Aaron Gray,0.899767,1.615385,2.515152,0.410256,0.207459,0.195804,0.547786,1.412587,2.256410,-0.622378,0.165501,1.577036e+06,180,0.508238,0.000000,0.562712,3433.016667,429,0.419580,8.002370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zhaire Smith,0.136364,0.545455,0.681818,0.545455,0.272727,0.090909,0.363636,0.545455,2.181818,0.772727,0.090909,3.012794e+06,10,0.377778,0.315789,0.666667,148.416667,22,0.454545,6.746212
Zhou Qi,0.136364,0.386364,0.522727,0.045455,0.045455,0.318182,0.227273,0.318182,0.590909,-1.568182,0.000000,9.310791e+05,30,0.235294,0.105263,0.666667,130.683333,44,0.681818,2.970076
Zion Williamson,2.560000,3.440000,6.000000,2.000000,0.640000,0.360000,2.360000,1.680000,21.600000,2.480000,0.960000,1.034991e+07,12,0.583333,0.428571,0.640449,668.133333,25,0.480000,26.725333


In [212]:
per_player_stats.to_excel('/Users/rickykuo/rickykuo81704@gmail.com - Google Drive/My Drive/Jupyter/NBA Data/per_player_stats.xlsx')